# **Análise de dados de crédito – Compreendendo a inadimplência do cliente**
Data Science



**Bibliotecas**

Pandas as pd 
seaborn as sns


# **1. Exploração de Dados**
Vamos explorar dados de crédito presentes neste neste link. Os dados estão no formato CSV e contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de default, que indica se um cliente é adimplente( default = 0 ), ou inadimplente ( default = 1 ), ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.

In [ ]:
import pandas as pd
df = pd.read_csv(
    'https://raw.githubusercontent.com/andre-marcos-perez/' +
    'ebac-course-utils/develop/dataset/credito.csv',
    na_values='na'
)
df.head(n=10)

*  Recorrendo a ferramentas visuais e bibliotecas especializadas, visamos oferecer uma compreensão refinada desse cenário complexo.

# **2.1. Estrutura**

* Estrutura de Dados Verificamos o formato do DataFrame e a quantidade de adimplentes e inadimplentes.

In [ ]:
df.shape # retorna uma tupla ( quantidade de linhas quantidade de colunas )

In [ ]:
df[df['default'] ==0]. shape

In [ ]:
df[df['default'] ==1]. shape

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f" A proporçao clientes adimplentes é de " +\
      f"{round(100* qtd_adimplentes/ qtd_total, 2)}%")
print(f" A proporçao clientes inadimplentes é de " +\
      f"{round(100* qtd_adimplentes/ qtd_total, 2)}%")

# **2.2. Schema**

*  Esquema de dados Examinamos os tipos de dados e as primeiras linhas do DataFrame.

In [ ]:
 df.head(n=5)

In [ ]:
df.dtypes

In [ ]:
 df.select_dtypes('object').describe().transpose()

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

#  Limpeza de Dados para uma Fundação Sólida:

A antessala de nossa análise envolveu a abordagem estratégica da limpeza de dados. A fim de assegurar a integridade e confiabilidade dos resultados, empregamos técnicas especializadas para sanar lacunas e imperfeições nos conjuntos de dados.

In [ ]:
 df.head()

* Podemos verificar quais colunas possuem dados faltantes.

Dados faltantes podem ser:

Vazios ( "" );
Nulos ( None );
Não disponíveis ou aplicaveis ( na , NA , etc.);
Não numérico ( nan , NaN , NAN , etc).

In [ ]:
df.isna().any()

* A função abaixo levanta algumas estatisticas sobre as colunas dos dados faltantes.

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col:
        {
          'quantidade': qtd,
          "porcentagem": round(100 * qtd/total, 2)
        }
      }
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [ ]:
 stats_dados_faltantes(df=df)

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

# **3. Transformação e limpeza de dados**
Agora que conhecemos melhor a natureza do nosso conjunto de dados, vamos conduzir uma atividade conhecida como data wrangling que consiste na transformação e limpeza dos dados do conjunto para que possam ser melhor analisados. Em especial, vamos remover:

Corrigir o schema das nossas colunas;
Remover os dados faltantes.

# **3.1. Correção de schema**
Na etapa de exploração, notamos que as colunas limite_credito e valor_transacoes_12m estavam sendo interpretadas como colunas categóricas ( dtype = object ).

In [ ]:
 df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

* Vamos criar uma função lambda para limpar os dados. Mas antes, vamos testar sua aplicação através do método funcional map :

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))
valores_originais = [
    '12.691,51',
    '8.256,96',
    '3.418,56',
    '3.313,03',
    '4.716,22'
]

In [ ]:
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

* Com a função lambda de limpeza pronta, basta aplica-la nas colunas de interesse.



In [ ]:
 df.dtypes

In [ ]:
df.select_dtypes('object').describe().transpose()

In [ ]:
 df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# **3.2. Remoção de dados faltantes**
Como o pandas está ciente do que é um dados faltante, a remoção das linhas problemáticas é trivial.

In [ ]:
df.dropna(inplace=True)

In [ ]:
 df.shape

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporção de adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporção de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")

print(f"A proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# **4. Visualização de dados**
Os dados estão prontos, vamos criar diversas visualizações para correlacionar variáveis explicativas com a variável resposta para buscar entender qual fator leva um cliente a inadimplencia. E para isso, vamos sempre comparar a base com todos os clientes com a base de adimplentes e inadimplentes. Começamos então importando os pacotes de visualização e separando os clientes adimplentes e inadimplentes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
 df_inadimplente = df[df['default'] == 1]

In [ ]:
 df.select_dtypes('object').head(n=5)


# **4.1. Visualizações categóricas**
Iniciamos a exploração dos padrões de inadimplência ao examinarmos a escolaridade dos clientes. Um achado significativo foi revelado: os adimplentes exibiam consistentemente níveis mais elevados de educação. Esta observação preliminar proporcionou uma base intrigante para análises posteriores.

In [ ]:
coluna = 'escolaridade'

titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

eixo = 0

max_y = 0

max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

   

    df_to_plot = dataframe[coluna].value_counts().reset_index()  # Cria um novo DataFrame com as contagens

    df_to_plot.columns = [coluna, 'frequencia_absoluta']

    df_to_plot.sort_values(by=[coluna], inplace=True)    

    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixos[eixo])

    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=df_to_plot[coluna], rotation=90)

    _, max_y_f = f.get_ylim()

    max_y = max_y_f if max_y_f > max_y else max_y

    f.set(ylim=(0, max_y))

    eixo += 1

   

figura.show()

* Salário Anual

Desbravando o terreno salarial, constatamos uma disparidade substancial entre adimplentes e inadimplentes. A média salarial anual dos clientes adimplentes destacou-se como significativamente superior, sugerindo uma correlação entre estabilidade financeira e adimplência.

In [ ]:
 coluna = 'salario_anual'
titulos = [
    'Salário Anual dos Clientes',
    'Salário Anual dos Clientes Adimplentes',
    'Salário Anual dos Clientes Inadimplentes'
]

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

   

    df_to_plot = dataframe[coluna].value_counts().reset_index()  # Cria um novo DataFrame com as contagens

    df_to_plot.columns = [coluna, 'frequencia_absoluta']

    df_to_plot.sort_values(by=[coluna], inplace=True)    

    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixos[eixo])

    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=df_to_plot[coluna], rotation=90)

    _, max_y_f = f.get_ylim()

    max_y = max_y_f if max_y_f > max_y else max_y

    f.set(ylim=(0, max_y))

    eixo += 1

   

figura.show()

# **4.2. Visualizações numéricas**
Um Olhar Sobre 12 Meses: Ao se debruçar sobre as transações ao longo de um período de 12 meses, emergiu uma narrativa temporal intrigante. Os adimplentes exibiram uma atividade transacional mais robusta, alimentando reflexões sobre a gestão financeira a longo prazo como fator determinante na inadimplência.

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

* Quantidade de Transações nos Últimos 12 Meses

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = [
 'Qtd. de Transações no Último Ano',
 'Qtd. de Transações no Último Ano de Adimplentes',
 'Qtd. de Transações no Último Ano de Inadimplentes'
]
eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)
for dataframe in [df, df_adimplente, df_inadimplente]:

   

    df_to_plot = dataframe[coluna].value_counts().reset_index()  # Cria um novo DataFrame com as contagens

    df_to_plot.columns = [coluna, 'frequencia_absoluta']

    df_to_plot.sort_values(by=[coluna], inplace=True)    

    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixos[eixo])

    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=df_to_plot[coluna], rotation=90)

    _, max_y_f = f.get_ylim()

    max_y = max_y_f if max_y_f > max_y else max_y

    f.set(ylim=(0, max_y))

    eixo += 1

   

figura.show()

* Valor das Transações nos Últimos 12 Meses

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = [
 'Valor das Transações no Último Ano',
 'Valor das Transações no Último Ano de Adimplentes',
 'Valor das Transações no Último Ano de Inadimplentes'
]
eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)
for dataframe in [df, df_adimplente, df_inadimplente]:

   

    df_to_plot = dataframe[coluna].value_counts().reset_index()  # Cria um novo DataFrame com as contagens

    df_to_plot.columns = [coluna, 'frequencia_absoluta']

    df_to_plot.sort_values(by=[coluna], inplace=True)    

    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixos[eixo])

    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    f.set_xticklabels(labels=df_to_plot[coluna], rotation=90)

    _, max_y_f = f.get_ylim()

    max_y = max_y_f if max_y_f > max_y else max_y

    f.set(ylim=(0, max_y))

    eixo += 1

   

figura.show()


In [ ]:
f = sns.relplot(
 x='valor_transacoes_12m',
 y='qtd_transacoes_12m',
 data=df,
 hue='default'
)
_ = f.set(
 title='Relação entre Valor e Quantidade de Transações no Último Ano',
 xlabel='Valor das Transações no Último Ano',
 ylabel='Quantidade das Transações no Último Ano'
 )

# **Proporções Reveladoras:**
Ativos Inadimplentes vs. Ativos Adimplentes: Ao consolidar nossas descobertas, uma análise de proporções revelou uma distinção sutil, mas crucial. Os clientes ativos inadimplentes compunham 16,07% da base total, enquanto os ativos adimplentes representavam 15,72%. Essa diferença aparentemente modesta indicou dinâmicas distintas que merecem uma atenção estratégica.